###  Autor: Lilian Rebeca Carrera Lemus 
No. Carnet: 20008077  


# Práctica No. 1 - Regresión lineal utilizando Tensorflow

### Descripción general:

La tarea práctica consiste en usar el mismo dataset del proyecto del curso pasado e implementar gradient descent aplicado a regresión lineal con este(usando la variable con mayor correlación elegida en el proyecto anterior). Realizar el entrenamiento con gradient descent usando Tensorflow, así mismo se debe usar Tensorboard para monitorear que el error disminuya conforme avanzan las iteraciones "en vivo".

In [1]:
import tensorflow as tf
import numpy as np 
import pandas as pd
import datetime as dt
%load_ext tensorboard

print(tf.__version__)

2.1.0


In [2]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Instructions for updating:
non-resource variables are not supported in the long term
Enabled compatitility to tf1.x


In [3]:
#Cargamos los datos para este proyecto
datos = np.load('proyecto_training_data.npy')
datos

array([[2.08500e+05, 7.00000e+00, 8.56000e+02, 8.00000e+00, 2.00300e+03,
        6.50000e+01],
       [1.81500e+05, 6.00000e+00, 1.26200e+03, 6.00000e+00, 1.97600e+03,
        8.00000e+01],
       [2.23500e+05, 7.00000e+00, 9.20000e+02, 6.00000e+00, 2.00100e+03,
        6.80000e+01],
       ...,
       [2.66500e+05, 7.00000e+00, 1.18800e+03, 9.00000e+00, 1.94100e+03,
        6.60000e+01],
       [1.42125e+05, 5.00000e+00, 1.07800e+03, 5.00000e+00, 1.95000e+03,
        6.80000e+01],
       [1.47500e+05, 5.00000e+00, 1.25600e+03, 6.00000e+00, 1.96500e+03,
        7.50000e+01]])

In [4]:
#Convertimos el set de datos que vamos a utlizar para entrenamiento a dataframe para que sea más fácil identificar cada característica y hacer conversiones
#Agregar nombres a las columnas de acuerdo a la información proporcionada
df_datos = pd.DataFrame(data = datos, columns = ["PrecioVenta", "CalidadMaterial","PCPerPiso", "THabitaciones", "AConstruccion", "PLFrente"])

#Conversion para los datos de calidad del material, el total de habitaciones y el año de contruscción, ya que son enteros
df_datos = df_datos.astype({"CalidadMaterial" : int, "THabitaciones" : int, "AConstruccion" : int})

#Información básica de dataframe
df_datos.describe()

,PrecioVenta,CalidadMaterial,PCPerPiso,THabitaciones,AConstruccion,PLFrente
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1201.000000
mean,180921.195890,6.099315,1162.626712,6.517808,1971.267808,70.049958
std,79442.502883,1.382997,386.587738,1.625393,30.202904,24.284752
min,34900.000000,1.000000,334.000000,2.000000,1872.000000,21.000000
25%,129975.000000,5.000000,882.000000,5.000000,1954.000000,59.000000
50%,163000.000000,6.000000,1087.000000,6.000000,1973.000000,69.000000
75%,214000.000000,7.000000,1391.250000,7.000000,2000.000000,80.000000
max,755000.000000,10.000000,4692.000000,14.000000,2010.000000,313.000000


##### Dividimos los datos de la siguiente manera:
80% de las filas serán datos de entrenamiento y 20% de las filas serán para pruebas.

In [5]:
#Datos de entrenamiento
d_entrenamiento = int(0.8 * len(datos))
df_entrenamiento = df_datos[:d_entrenamiento]

#datos de validacion
df_validacion = df_datos[d_entrenamiento:]

print('Longitud del data set completo: ',len(df_datos))
print('Longitud del data set de entrenamiento: ',len(df_entrenamiento))
print('Longitud del data set de validación: ',len(df_validacion))

Longitud del data set completo:  1460
Longitud del data set de entrenamiento:  1168
Longitud del data set de validación:  292


In [6]:
#Calculamos el coeficiente de correlación
df_correlacion = df_entrenamiento.corr()
df_correlacion

,PrecioVenta,CalidadMaterial,PCPerPiso,THabitaciones,AConstruccion,PLFrente
PrecioVenta,1.000000,0.793990,0.616289,0.564707,0.534171,0.363292
CalidadMaterial,0.793990,1.000000,0.469411,0.444293,0.567313,0.242420
PCPerPiso,0.616289,0.469411,1.000000,0.409318,0.292074,0.417388
THabitaciones,0.564707,0.444293,0.409318,1.000000,0.122911,0.341005
AConstruccion,0.534171,0.567313,0.292074,0.122911,1.000000,0.121474
PLFrente,0.363292,0.242420,0.417388,0.341005,0.121474,1.000000


### Elección de Variable

Se ha decidido elegir la variable de Calidad del Material, ya que de acuerdo a su coeficiente de correlación, es la variable que mejor explica o estima el precio de venta de las casas(Variable Dependiente)

## Modelo de Regresión Lineal y Gradient Descent

Este modelo lo que trata es de predecir los precios de casas utilizando un modelo de regresión lineal simple. Este modelo de regresión lineal busca encontrar la ecuación lineal que minimice el error cuadrado medio (MSE).

La ecuación lineal para el modelo es la siguiente:

$h(x) = wx + b$

*donde:*

$x$ : Variable independiente  
$w$ : Pendiente  
$b$ : Intercepto  

y la función de costo a minimizar es:

$C(w,b) = \frac{1}{2} \sum_{i=1}^{n} (y_i − h(x_i))^2$

*donde:*

$y_i$ : Valor real de cada dato en el dataset  
$h(x_{i})$ : Valor predecido por nuestro modelo  

Para implementar Gradient Descent aplicado a regresión el modelo de Regresión Lineal se entrenará utilizando la función de costo indicada. El algoritmo de gradient descent calcula la derivada parcial para cada uno de los parámetros (w,b) de la función de costo y en cada iteración los valores de $m$ y $b$ son actualizados iterativamente hasta minimizar el error.

### Implementación de Modelo de Regresión Lineal utilizando Tensorflow

In [7]:
#Aplicando reutilización de código por medio del uso de programación orientada a objetos para crear el modelo
class ModeloLineal:    
    def __init__(self):
         # Vector de parametros, para pendiente e intercepto
        self.parametros = tf.get_variable("parametros",dtype=tf.float32,shape=[1,2],initializer=tf.zeros_initializer())
    
    def __call__(self,x):
        #Calculamos "y" estimada
        return tf.matmul(self.parametros,x, name="MatMul")

    def actualizar(self,x,y,learning_rate):
        with tf.name_scope("modelo"):
            prediccion = self(x)
        
        with tf.name_scope("error"):
            #Calculo del error MSE mediante la fórmula de costo
            error = 1/2 * tf.reduce_mean(tf.math.square(y - prediccion))
            #Scalar para utilizar en tensorboard
            error_summary = tf.summary.scalar("error_summary", error)
                                     
        #Actualizamos los parametros por medio de Gradient Descent
        gradiente = tf.gradients(error, [self.parametros], name="gradientes")
        actualizacion_parametros = tf.assign(self.parametros, self.parametros -learning_rate*gradiente[0], name="actualizar")
        
        return actualizacion_parametros, error, error_summary

In [8]:
#Aplicando reutilización de código por medio del uso de programación orientada a objetos p/definir la fun. de entrenamiento
def entrenamiento(x, y, lr, epochs, imprimir_error):
    # Definimos el string de configuración para cada experimiento
    string_conf = './logs/'+ dt.datetime.now().strftime("%Y%m%d-%H%M%S") +"_lr="+str(lr)+ "_epochs="+str(epochs)
    
    tf.reset_default_graph()
    g = tf.Graph()  #definición del grafo a usar
    
    with g.as_default():
        modelo = ModeloLineal()
        #Transformamos x para poder utilizarla en el modelo
        x = np.array([x, np.ones_like(x)], dtype = "float64")
        x_fil, x_col = np.shape(x)
        # Definimos las entradas del grafo, los tensores para x y para y
        tensor_y = tf.placeholder(float,[len(y)], name="Y")
        tensor_x = tf.placeholder(float,[x_fil,x_col], name="X")
        
        # Realizamos el entrenamiento mediante el metodo actualizar
        actualizar_parametros = modelo.actualizar(tensor_x, tensor_y, lr)
        
        # Escribimos el archivo para tensorboard
        writer = tf.summary.FileWriter(string_conf, g)
        
        with tf.train.MonitoredSession() as session:
            feed_dict = {tensor_x:x, tensor_y:y}
            for i in range(1, epochs+1):   
                # Ejecutamos el entrenamiento
                entrenamiento = session.run(actualizar_parametros, feed_dict = feed_dict)
            
                if (i)%imprimir_error == 0:
                    # Calculamos los parametros para cada epoch que se va a imprimir
                    parametros = session.run(modelo.parametros, feed_dict = feed_dict)
                    # Agregamos un escalar para monitorear el MSE posteriormente en tensorboard
                    writer.add_summary(entrenamiento[2], i)
                    print("No. de Iteración:",i,", Parametros del modelo: ", parametros,", Error del modelo:",entrenamiento[1])
                    print("_______________________________________________________________________________________________________")
                    
            writer.close()

### Grafo del Modelo implementado

<img src="./Imagenes/Grafo del Modelo.jpg">

### Experimentos utilizando el Modelo Generado

Es necesario realizar varios experimentos para poder encontrar el valor óptimo de los hiper-párametros del modelo, como los son en este caso el learning rate y el número de epochs. Debido a que no hay una receta para poder determinar estos hiper-párametros de entrenamiento para el modelo, nos vemos en la necesidad de experimentar con los mismos. 

Para este modelo la rapidez con el que el modelo alcanza el error mínimo depende del learning rate (tamaño del paso), lo cual condiciona también el número de epochs (número de pasos) necesario para llegar al mínimo.


In [9]:
#Obtenemos los valores para "x" y ""y  del data frame de entrenamiento para usarlos en los experimentos
y = df_entrenamiento["PrecioVenta"]
x = df_entrenamiento["CalidadMaterial"]

### Experimento No.1

En este primer experimento utilizaremos un learning rate pequeño, por lo cual esto condiciona la necesidad de utilizar un número de epochs mayor para poder llegar a un mínimo


In [10]:
#lr = 0.001, epochs= 15000
entrenamiento(x, y, 0.001, 15000,3000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
No. de Iteración: 3000 , Parametros del modelo:  [[31716.578 -8760.742]] , Error del modelo: 1333491700.0
_______________________________________________________________________________________________________
No. de Iteración: 6000 , Parametros del modelo:  [[ 33530.043 -20374.766]] , Error del modelo: 1287352200.0
_______________________________________________________________________________________________________
No. de Iteración: 9000 , Parametros del modelo:  [[ 35103.37  -30450.777]] , Error del modelo: 1252623200.0
_______________________________________________________________________________________________________
No. de Iteración: 12000 , Parametros del modelo:  [[ 36468.4   -39192.555]] , Error del modelo: 1226482600.0
_______________________________________________________________________________________________________
No. de Iteración: 15000 , Paramet

<img src="./Imagenes/Experimento1.png">

### Experimento No.2

Debido a que el error está disminuyendo pero no converge en el experimento1, en este segundo experimento utilizaremos un learning rate más grande, para que los pasos sean más grandes y mantendremos el número de epochs igual para poder llegar a un mínimo. Esperamos que ahora el error si tienda a disminuir más rápido, ya que daremos pasos más grandes para llegar al mínimo.


In [11]:
#lr = 0.01, epochs= 15000
entrenamiento(x, y, 0.01, 15000,3000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
No. de Iteración: 3000 , Parametros del modelo:  [[ 41599.26 -72051.84]] , Error del modelo: 1161388700.0
_______________________________________________________________________________________________________
No. de Iteración: 6000 , Parametros del modelo:  [[ 44491.09 -90571.84]] , Error del modelo: 1147760500.0
_______________________________________________________________________________________________________
No. de Iteración: 9000 , Parametros del modelo:  [[ 45189.555 -95044.984]] , Error del modelo: 1146965500.0
_______________________________________________________________________________________________________
No. de Iteración: 12000 , Parametros del modelo:  [[ 45358.25 -96125.35]] , Error del modelo: 1146919000.0
_______________________________________________________________________________________________________
No. de Iteración: 15000 , Parametros 

<img src="./Imagenes/Experimento2.png">

### Experimento No.3

Pudimos observar que en el experimento anterior el error disminuyó considereblemente, por lo cual en este tercer experimento mantendremos el valor del learning rate y aumentaremos el número de epochs para dar más pasos. Esperamos que el error disminuya más, ya que daremos más pasos para llegar al mínimo.


In [12]:
#lr = 0.01, epochs= 25000
entrenamiento(x, y, 0.01, 25000,5000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
No. de Iteración: 5000 , Parametros del modelo:  [[ 43933.26  -86999.375]] , Error del modelo: 1149093200.0
_______________________________________________________________________________________________________
No. de Iteración: 10000 , Parametros del modelo:  [[ 45273.457 -95582.31 ]] , Error del modelo: 1146935400.0
_______________________________________________________________________________________________________
No. de Iteración: 15000 , Parametros del modelo:  [[ 45398.973 -96386.13 ]] , Error del modelo: 1146916400.0
_______________________________________________________________________________________________________
No. de Iteración: 20000 , Parametros del modelo:  [[ 45410.65  -96460.945]] , Error del modelo: 1146916000.0
_______________________________________________________________________________________________________
No. de Iteración: 25000 , Par

<img src="./Imagenes/Experimento3.png">

### Experimento No.4

Debido a que en el tercer experimento aumentando el número de epochs el error bajó muy poco, en este cuarto experimento  aumentaremos el valor de learning rate y bajaremos el número de epochs. Con esto se espera que dando pasos más grandes necesitemos de menos epochs para llegar al mínimo.

In [13]:
#lr = 0.02, epochs= 10000
entrenamiento(x, y, 0.02, 10000,2000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
No. de Iteración: 2000 , Parametros del modelo:  [[ 43038.633 -81269.945]] , Error del modelo: 1152529300.0
_______________________________________________________________________________________________________
No. de Iteración: 4000 , Parametros del modelo:  [[ 45055.17 -94184.34]] , Error del modelo: 1147043000.0
_______________________________________________________________________________________________________
No. de Iteración: 6000 , Parametros del modelo:  [[ 45358.34 -96125.94]] , Error del modelo: 1146918900.0
_______________________________________________________________________________________________________
No. de Iteración: 8000 , Parametros del modelo:  [[ 45403.918 -96417.81 ]] , Error del modelo: 1146916200.0
_______________________________________________________________________________________________________
No. de Iteración: 10000 , Parametros

<img src="./Imagenes/Experimento4.png">

### Experimento No.5

Debido a que en el cuarto experimento el error bajó muy poco, en este quinto experimento aumentaremos un poco más el valor de learning rate y mantendremos el número de epochs. Con esto se espera que disminuya el error.

In [14]:
#lr = 0.03, epochs= 10000
entrenamiento(x, y, 0.03, 10000,2000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
No. de Iteración: 2000 , Parametros del modelo:  [[ 44492.344 -90579.875]] , Error del modelo: 1147759700.0
_______________________________________________________________________________________________________
No. de Iteración: 4000 , Parametros del modelo:  [[ 45358.42  -96126.445]] , Error del modelo: 1146918800.0
_______________________________________________________________________________________________________
No. de Iteración: 6000 , Parametros del modelo:  [[ 45408.88 -96449.6 ]] , Error del modelo: 1146916100.0
_______________________________________________________________________________________________________
No. de Iteración: 8000 , Parametros del modelo:  [[ 45411.562 -96466.78 ]] , Error del modelo: 1146916100.0
_______________________________________________________________________________________________________
No. de Iteración: 10000 , Parametr

<img src="./Imagenes/Experimento5.png">

### Experimento No.6

Debido a que en el experimento anterior el error bajo rapidamente pero a partir del epoch 600 se mantuvo el mismo valor, corroborando la hipotesis esperada en el experimento anterior. En este sexto experimento aumentaremos un poco más el valor de learning rate y mantendremos el número de epochs. Con esto se esperamos que dando pasos más grandes necesitemos aún de menos epochs para llegar al mínimo.

In [15]:
#lr = 0.04, epochs= 10000
entrenamiento(x, y, 0.04, 10000,2000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
No. de Iteración: 2000 , Parametros del modelo:  [[ 45055.81 -94188.43]] , Error del modelo: 1147043200.0
_______________________________________________________________________________________________________
No. de Iteración: 4000 , Parametros del modelo:  [[ 45403.965 -96418.125]] , Error del modelo: 1146916700.0
_______________________________________________________________________________________________________
No. de Iteración: 6000 , Parametros del modelo:  [[ 45411.676 -96467.51 ]] , Error del modelo: 1146916000.0
_______________________________________________________________________________________________________
No. de Iteración: 8000 , Parametros del modelo:  [[ 45411.676 -96467.51 ]] , Error del modelo: 1146916000.0
_______________________________________________________________________________________________________
No. de Iteración: 10000 , Parametr

<img src="./Imagenes/Experimento6.png">

### Experimento No.7

En el experimento anterior nuevamente el error disminuyó rapidamente y se mantuvo a partir de la iteración 600, por lo cual en este séptimo experimento aumentaremos un poco más el valor de learning rate y mantendremos el número de iteraciones. Esperamos encontrar el número óptimo para el larning rate, es decir hasta donde podríamos aumentar el tamaño de los pasos para llegar al mínimo.

In [16]:
#lr = 0.05, epochs= 10000
entrenamiento(x, y, 0.05, 10000,2000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
No. de Iteración: 2000 , Parametros del modelo:  [[-41193584.   -6534861.5]] , Error del modelo: 3.4653553e+16
_______________________________________________________________________________________________________
No. de Iteración: 4000 , Parametros del modelo:  [[-5.7403527e+10 -8.9634847e+09]] , Error del modelo: 6.7145005e+22
_______________________________________________________________________________________________________
No. de Iteración: 6000 , Parametros del modelo:  [[-7.9905644e+13 -1.2476931e+13]] , Error del modelo: 1.3010384e+29
_______________________________________________________________________________________________________
No. de Iteración: 8000 , Parametros del modelo:  [[-1.1122632e+17 -1.7367456e+16]] , Error del modelo: inf
_______________________________________________________________________________________________________
No. de Itera

<img src="./Imagenes/Experimento7.png">

### Experimento No.8

Como pudimos observar en el experimento anterior el learning rate de 0.05 causa divergencia, por lo cual en este octavo experimento bajaremos el valor del learning rate hasta lo máximo que pudimos aumentarlo antes de divergiera y disminuiremos el número de iteraciones. Esperamos encontrar el número de iteraciones óptimas, esperando que sean menos debido a que tendremos un larning rate grande.

In [17]:
#lr = 0.04, epochs= 6000
entrenamiento(x, y, 0.04, 6000,1000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
No. de Iteración: 1000 , Parametros del modelo:  [[ 43040.75 -81283.51]] , Error del modelo: 1152530000.0
_______________________________________________________________________________________________________
No. de Iteración: 2000 , Parametros del modelo:  [[ 45055.81 -94188.43]] , Error del modelo: 1147043200.0
_______________________________________________________________________________________________________
No. de Iteración: 3000 , Parametros del modelo:  [[ 45358.492 -96126.9  ]] , Error del modelo: 1146918700.0
_______________________________________________________________________________________________________
No. de Iteración: 4000 , Parametros del modelo:  [[ 45403.965 -96418.125]] , Error del modelo: 1146916700.0
_______________________________________________________________________________________________________
No. de Iteración: 5000 , Parametros 

<img src="./Imagenes/Experimento8.png">

### Experimento No.9

Como pudimos observar en el experimento anterior efectivamente necesitamos menos iteraciones para llegar al mínimo, por lo cual en este noveno experimento bajaremos el número de iteraciones. Esperamos encontrar el número de iteraciones óptimas, esperando que sean cada vez menos.

In [18]:
#lr = 0.04, epochs= 5000
entrenamiento(x, y, 0.04, 5000,1000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
No. de Iteración: 1000 , Parametros del modelo:  [[ 43040.75 -81283.51]] , Error del modelo: 1152530000.0
_______________________________________________________________________________________________________
No. de Iteración: 2000 , Parametros del modelo:  [[ 45055.81 -94188.43]] , Error del modelo: 1147043200.0
_______________________________________________________________________________________________________
No. de Iteración: 3000 , Parametros del modelo:  [[ 45358.492 -96126.9  ]] , Error del modelo: 1146918700.0
_______________________________________________________________________________________________________
No. de Iteración: 4000 , Parametros del modelo:  [[ 45403.965 -96418.125]] , Error del modelo: 1146916700.0
_______________________________________________________________________________________________________
No. de Iteración: 5000 , Parametros 

<img src="./Imagenes/Experimento9.png">

### Experimento No.10

Como pudimos observar en el experimento anterior efectivamente necesitamos menos iteraciones para llegar al mínimo, por lo cual en este decimo experimento bajaremos el número de iteraciones aún más. Esperamos encontrar el número de iteraciones óptimas, es decir cuál es el menor número de iteraciones que podremos usar para llegar al mínimo.

In [19]:
#lr = 0.04, epochs= 4000
entrenamiento(x, y, 0.04, 4000,1000)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
No. de Iteración: 1000 , Parametros del modelo:  [[ 43040.75 -81283.51]] , Error del modelo: 1152530000.0
_______________________________________________________________________________________________________
No. de Iteración: 2000 , Parametros del modelo:  [[ 45055.81 -94188.43]] , Error del modelo: 1147043200.0
_______________________________________________________________________________________________________
No. de Iteración: 3000 , Parametros del modelo:  [[ 45358.492 -96126.9  ]] , Error del modelo: 1146918700.0
_______________________________________________________________________________________________________
No. de Iteración: 4000 , Parametros del modelo:  [[ 45403.965 -96418.125]] , Error del modelo: 1146916700.0
_______________________________________________________________________________________________________


<img src="./Imagenes/Experimento10.png">

### Experimento No.11

Como pudimos observar en el experimento anterior con 4000 iteraciones ya no pudimos llegar al mínimo, por lo cual en este decimo experimento  umentaremos el número de iteraciones un poco. Esperamos encontrar el número de iteraciones óptimas, es decir cuál es el menor número de iteraciones que podremos usar para llegar al mínimo.

In [20]:
#lr = 0.04, epochs= 4500
entrenamiento(x, y, 0.04, 4500,500)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
No. de Iteración: 500 , Parametros del modelo:  [[ 39293.85  -57287.406]] , Error del modelo: 1184288600.0
_______________________________________________________________________________________________________
No. de Iteración: 1000 , Parametros del modelo:  [[ 43040.75 -81283.51]] , Error del modelo: 1152530000.0
_______________________________________________________________________________________________________
No. de Iteración: 1500 , Parametros del modelo:  [[ 44492.97 -90583.87]] , Error del modelo: 1147759400.0
_______________________________________________________________________________________________________
No. de Iteración: 2000 , Parametros del modelo:  [[ 45055.81 -94188.43]] , Error del modelo: 1147043200.0
_______________________________________________________________________________________________________
No. de Iteración: 2500 , Parametros del

<img src="./Imagenes/Experimento11.png">

Como se puede observar en este experimento, utilizando un número de iteraciones de 4500 no pudimos llegar al error mínimo.

### Conclusiones Generales:

* Un learning rate de 0.04 es cercano al máximo permitido antes de que el algoritmo de Gradient Descent empiece a diverger.
* Debido a que 0.04 es un learning rate que permite tomar pasos más grandes, son necesarias menos iteraciones (epochs) para llegar al mínimo de la función de costo.
* El mejor experimento es el número nueve, ya que cuenta con la combinación óptima de larning rate(tamaño del paso) y número de iteraciones(epochs) para llegar al error mínimo, esta combinación es un larning rate= 0.04 y un número de epochs=5000.

Gráfico general en Tensorboard de la ejecución de los experimentos, excluyendo el experimento No.7 ya que tiende al infinito (el experimento No.1 tampoco se aprecia en la imagen debido a la escala).

<img src="./Imagenes/GeneralExperimentos.png">


#### Integración de Tensorboard en el Notebook

In [23]:
%tensorboard --logdir  ./logs --host localhost

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 21144.

In [24]:
%tensorboard --logdir  ./logs --host localhost

Reusing TensorBoard on port 6006 (pid 3680), started 0:01:07 ago. (Use '!kill 3680' to kill it.)